In [ ]:
# 📦 Import libraries
import os
import torch
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.ops import box_iou

In [ ]:
# 🧾 Dataset class
class MicroplasticDetectionDataset(Dataset):
    def __init__(self, image_dir, annotation_file, transform=None):
        self.image_dir = image_dir
        self.annotations = pd.read_csv(annotation_file)
        self.image_files = self.annotations["filename"].unique()
        self.transform = transform
        self.class_dict = {"Microplastic": 1}

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        filename = self.image_files[idx]
        image_path = os.path.join(self.image_dir, filename)
        image = Image.open(image_path).convert("RGB")

        boxes = []
        labels = []

        records = self.annotations[self.annotations["filename"] == filename]
        for _, row in records.iterrows():
            boxes.append([row["xmin"], row["ymin"], row["xmax"], row["ymax"]])
            labels.append(self.class_dict.get(row["class"], 0))

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
            "image_id": torch.tensor([idx])
        }

        if self.transform:
            image = self.transform(image)
        else:
            image = ToTensor()(image)

        return image, target

In [ ]:
# 📁 Path configuration
train_img_dir = "/kaggle/input/microplastic-dataset-for-computer-vision/train"
valid_img_dir = "/kaggle/input/microplastic-dataset-for-computer-vision/valid"
train_csv_path = os.path.join(train_img_dir, "_annotations.csv")
valid_csv_path = os.path.join(valid_img_dir, "_annotations.csv")


In [ ]:
# 📚 Dataset & Dataloader
train_dataset = MicroplasticDetectionDataset(train_img_dir, train_csv_path)
valid_dataset = MicroplasticDetectionDataset(valid_img_dir, valid_csv_path)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


In [ ]:
# 🧠 Model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = fasterrcnn_resnet50_fpn(weights=None, weights_backbone=None, num_classes=2)
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)


In [ ]:
# 🔁 Training + validation loop
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"🧠 Epoch {epoch + 1}/{num_epochs} - Train Loss: {epoch_loss:.4f}")

    # Validasi jumlah deteksi
    def evaluate_on_validation(model, data_loader, device, score_thresh=0.5):
        model.eval()
        total_preds, total_gts = 0, 0
        with torch.no_grad():
            for images, targets in data_loader:
                images = [img.to(device) for img in images]
                outputs = model(images)
                for output, target in zip(outputs, targets):
                    preds = output["boxes"][output["scores"] > score_thresh]
                    total_preds += len(preds)
                    total_gts += len(target["boxes"])
        model.train()
        return total_preds, total_gts

    val_preds, val_gts = evaluate_on_validation(model, valid_loader, device)
    print(f"🔍 Validation - Predictions: {val_preds}, Ground Truths: {val_gts}")

    # Save model
    torch.save(model.state_dict(), f"fasterrcnn_microplastic_epoch{epoch+1}.pth")
    print(f"✅ Model saved: fasterrcnn_microplastic_epoch{epoch+1}.pth")


In [ ]:
# 📊 Evaluasi detail per gambar
def compute_metrics_per_image(pred_boxes, gt_boxes, iou_thresh=0.5):
    if len(pred_boxes) == 0 and len(gt_boxes) == 0:
        return 1, 1, 1, 1
    if len(pred_boxes) == 0 or len(gt_boxes) == 0:
        return 0, 0, 0, 0

    ious = box_iou(pred_boxes, gt_boxes)
    matched = set()
    tp = 0
    for i in range(len(pred_boxes)):
        max_iou = 0
        match_idx = -1
        for j in range(len(gt_boxes)):
            if j in matched:
                continue
            if ious[i, j] > max_iou:
                max_iou = ious[i, j]
                match_idx = j
        if max_iou >= iou_thresh:
            tp += 1
            matched.add(match_idx)

    fp = len(pred_boxes) - tp
    fn = len(gt_boxes) - tp
    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)
    mean_iou = ious.mean().item() if ious.numel() > 0 else 0
    return precision, recall, f1, mean_iou


In [ ]:
# 🧾 Load validation annotation
val_annotations = pd.read_csv(valid_csv_path)

In [ ]:
# 📦 Evaluasi model
results = []
model.eval()

for fname in val_annotations["filename"].unique():
    image_path = os.path.join(valid_img_dir, fname)
    image = Image.open(image_path).convert("RGB")
    img_tensor = ToTensor()(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)[0]

    pred_boxes = output["boxes"][output["scores"] > 0.5].cpu()
    pred_scores = output["scores"][output["scores"] > 0.5].cpu()
    gt = val_annotations[val_annotations["filename"] == fname]
    gt_boxes = torch.tensor(gt[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float)

    precision, recall, f1, iou = compute_metrics_per_image(pred_boxes, gt_boxes)

    results.append({
        "filename": fname,
        "actual_count": len(gt_boxes),
        "predicted_count": len(pred_boxes),
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "mean_iou": iou
    })

In [ ]:
# 💾 Simpan hasil evaluasi
results_df = pd.DataFrame(results)
results_df.to_csv("microplastic_detection_results.csv", index=False)
print("📁 Evaluation results saved to: microplastic_detection_results.csv")


In [ ]:
# 📈 Rata-rata metrik keseluruhan
print("\n📊 Summary of Detection Metrics:")
print(f"Average Precision: {results_df['precision'].mean():.4f}")
print(f"Average Recall:    {results_df['recall'].mean():.4f}")
print(f"Average F1 Score:  {results_df['f1_score'].mean():.4f}")
print(f"Average IoU:       {results_df['mean_iou'].mean():.4f}")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
from PIL import Image
import pandas as pd
from torchvision.transforms import ToTensor

class MicroplasticCSVDetectionDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, csv_path, transforms=None):
        self.img_dir = img_dir
        self.df = pd.read_csv(csv_path)
        self.transforms = transforms
        self.image_filenames = self.df["filename"].unique()

    def __getitem__(self, idx):
        img_filename = self.image_filenames[idx]
        img_path = os.path.join(self.img_dir, img_filename)
        img = Image.open(img_path).convert("RGB")

        # Get all rows in the CSV that match this image
        records = self.df[self.df["filename"] == img_filename]
        
        boxes = []
        labels = []

        for _, row in records.iterrows():
            xmin = row["xmin"]
            ymin = row["ymin"]
            xmax = row["xmax"]
            ymax = row["ymax"]
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(int(row["class"]) + 1)  # Background is 0, microplastic is 1

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
            "image_id": torch.tensor([idx])
        }

        if self.transforms:
            img = self.transforms(img)
        else:
            img = ToTensor()(img)

        return img, target

    def __len__(self):
        return len(self.image_filenames)


In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Custom Dataset
class MicroplasticCSVDetectionDataset(Dataset):
    def __init__(self, img_dir, csv_path, transforms=None):
        self.img_dir = img_dir
        self.df = pd.read_csv(csv_path)
        self.transforms = transforms
        self.image_filenames = self.df["filename"].unique()
        self.class_map = {"Microplastic": 1}  # Mapping string to int

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_filename = self.image_filenames[idx]
        img_path = os.path.join(self.img_dir, img_filename)
        img = Image.open(img_path).convert("RGB")

        records = self.df[self.df["filename"] == img_filename]

        boxes = []
        labels = []

        for _, row in records.iterrows():
            xmin = row["xmin"]
            ymin = row["ymin"]
            xmax = row["xmax"]
            ymax = row["ymax"]
            class_name = row["class"]
            class_id = self.class_map.get(class_name, 0)
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(class_id)

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
            "image_id": torch.tensor([idx])
        }

        if self.transforms:
            img = self.transforms(img)
        else:
            img = ToTensor()(img)

        return img, target

# Paths
train_dir = "/kaggle/input/microplastic-dataset-for-computer-vision/train"
valid_dir = "/kaggle/input/microplastic-dataset-for-computer-vision/valid"
train_csv = os.path.join(train_dir, "_annotations.csv")
valid_csv = os.path.join(valid_dir, "_annotations.csv")

# Datasets
train_dataset = MicroplasticCSVDetectionDataset(train_dir, train_csv)
valid_dataset = MicroplasticCSVDetectionDataset(valid_dir, valid_csv)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Model
model = fasterrcnn_resnet50_fpn(num_classes=2)  # 1 class + background
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

# Training
model.train()
for epoch in range(5):
    total_loss = 0
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

# Put model in eval mode
model.eval()

# Get a batch of validation data
images, targets = next(iter(valid_loader))
images = [img.to(device) for img in images]

# Inference
with torch.no_grad():
    predictions = model(images)

# Visualize predictions for 1 image
image = images[0].cpu()
pred = predictions[0]
boxes = pred['boxes'].cpu()
scores = pred['scores'].cpu()
labels = pred['labels'].cpu()

# Plot
plt.figure(figsize=(8, 8))
img = F.to_pil_image(image)
plt.imshow(img)

for box, score, label in zip(boxes, scores, labels):
    if score > 0.5:  # Confidence threshold
        x1, y1, x2, y2 = box
        plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                          edgecolor='red', facecolor='none', linewidth=2))
        plt.text(x1, y1, f'Microplastic: {score:.2f}', color='white',
                 bbox=dict(facecolor='red', alpha=0.5))

plt.axis('off')
plt.title('Predictions')
plt.show()


In [ ]:
torch.save(model.state_dict(), "fasterrcnn_microplastic.pth")


In [ ]:
model.load_state_dict(torch.load("fasterrcnn_microplastic.pth"))
model.eval()


In [ ]:
from PIL import Image

def predict_image(image_path):
    model.eval()
    img = Image.open(image_path).convert("RGB")
    img_tensor = ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)[0]

    return output

output = predict_image("/kaggle/input/microplastic-dataset-for-computer-vision/valid/a--3-_jpg.rf.8248ba99e3b3ae254d1723b674f7fd99.jpg")
print(output)


In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torch

# Load model
model = fasterrcnn_resnet50_fpn(num_classes=2)
model.load_state_dict(torch.load("fasterrcnn_microplastic.pth"))  # Update path if needed
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
from torchvision.transforms import ToTensor
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

def predict_and_plot(image_path, threshold=0.5):
    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")
    image_tensor = ToTensor()(image).unsqueeze(0).to(device)

    # Run prediction
    with torch.no_grad():
        outputs = model(image_tensor)

    outputs = outputs[0]
    boxes = outputs['boxes'].cpu()
    scores = outputs['scores'].cpu()
    labels = outputs['labels'].cpu()

    # Plot
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    ax = plt.gca()

    for box, score in zip(boxes, scores):
        if score > threshold:
            x1, y1, x2, y2 = box
            ax.add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                       linewidth=2, edgecolor='lime', facecolor='none'))
            ax.text(x1, y1, f'{score:.2f}', color='white',
                    bbox=dict(facecolor='lime', edgecolor='none', alpha=0.7))

    plt.axis("off")
    plt.title("Predicted Microplastics")
    plt.show()


In [ ]:
predict_and_plot("/kaggle/input/microplastic-dataset-for-computer-vision/valid/a--23-_jpg.rf.1ab5e302030f3bb3c08981ca42a8e631.jpg")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import torch
from torchvision.transforms import ToTensor

def visualize_prediction_vs_actual(image_path, annotation_df, model, threshold=0.5):
    # Extract filename
    filename = image_path.split("/")[-1]

    # Load image
    image = Image.open(image_path).convert("RGB")
    image_tensor = ToTensor()(image).unsqueeze(0).to(device)

    # Predict with model
    model.eval()
    with torch.no_grad():
        prediction = model(image_tensor)[0]

    pred_boxes = prediction['boxes'].cpu()
    pred_scores = prediction['scores'].cpu()
    pred_boxes = pred_boxes[pred_scores > threshold]

    # Get ground truth from CSV
    gt_boxes = annotation_df[annotation_df["filename"] == filename]
    
    # Plotting
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(image)

    # Draw ground truth (green)
    for _, row in gt_boxes.iterrows():
        xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]
        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                 linewidth=2, edgecolor='lime', facecolor='none')
        ax.add_patch(rect)

    # Draw predictions (red)
    for box in pred_boxes:
        x1, y1, x2, y2 = box
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                 linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)

    # Count
    actual_count = len(gt_boxes)
    predicted_count = len(pred_boxes)

    plt.title(f"Actual: {actual_count} vs Predicted: {predicted_count}", fontsize=16)
    plt.axis("off")
    plt.show()


In [ ]:
# Example: visualize for one image from validation set
image_path = "/kaggle/input/microplastic-dataset-for-computer-vision/valid/a--46-_jpg.rf.f2fc88b781d9457f65bfee1a29f6ca49.jpg"
annotation_csv = pd.read_csv("/kaggle/input/microplastic-dataset-for-computer-vision/valid/_annotations.csv")

visualize_prediction_vs_actual(image_path, annotation_csv, model)


In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision.transforms import ToTensor
from torchvision.ops import box_iou

# Load your annotation CSV
valid_csv_path = "/kaggle/input/microplastic-dataset-for-computer-vision/valid/_annotations.csv"
annotation_df = pd.read_csv(valid_csv_path)

# Folder where your images are
valid_images_dir = "/kaggle/input/microplastic-dataset-for-computer-vision/valid"

# Get unique image filenames
image_files = annotation_df["filename"].unique()

# Evaluation function
def evaluate_detection(pred_boxes, gt_boxes, iou_threshold=0.5):
    if len(pred_boxes) == 0 and len(gt_boxes) == 0:
        return 1, 1, 1, 1
    if len(pred_boxes) == 0 or len(gt_boxes) == 0:
        return 0, 0, 0, 0

    ious = box_iou(pred_boxes, gt_boxes)
    matched_gt = set()
    tp = 0

    for i in range(len(pred_boxes)):
        max_iou = 0
        max_j = -1
        for j in range(len(gt_boxes)):
            if j in matched_gt:
                continue
            if ious[i, j] > max_iou:
                max_iou = ious[i, j]
                max_j = j
        if max_iou >= iou_threshold:
            tp += 1
            matched_gt.add(max_j)

    fp = len(pred_boxes) - tp
    fn = len(gt_boxes) - tp

    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)
    mean_iou = ious.mean().item() if ious.numel() > 0 else 0
    return precision, recall, f1, mean_iou

# Run evaluation on all images
results = []
model.eval()

for filename in image_files:
    img_path = os.path.join(valid_images_dir, filename)
    image = Image.open(img_path).convert("RGB")
    img_tensor = ToTensor()(image).unsqueeze(0).to(device)

    with torch.no_grad():
        prediction = model(img_tensor)[0]

    # Get predicted boxes
    pred_boxes = prediction['boxes'][prediction['scores'] > 0.5].cpu()

    # Get ground truth boxes for that image
    gt = annotation_df[annotation_df["filename"] == filename]
    gt_boxes = torch.tensor(gt[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float)

    precision, recall, f1, iou = evaluate_detection(pred_boxes, gt_boxes)

    results.append({
        "filename": filename,
        "actual_count": len(gt_boxes),
        "predicted_count": len(pred_boxes),
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "mean_iou": iou
    })

# Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("microplastic_detection_metrics.csv", index=False)
print("Evaluation saved to microplastic_detection_metrics.csv")


In [ ]:

import os
import torch
import pandas as pd
from PIL import Image
from torchvision.transforms import ToTensor
from torchvision.ops import box_iou

# Path to validation data
valid_csv_path = "/kaggle/input/microplastic-dataset-for-computer-vision/valid/_annotations.csv"
valid_images_dir = "/kaggle/input/microplastic-dataset-for-computer-vision/valid"
annotation_df = pd.read_csv(valid_csv_path)

image_files = annotation_df["filename"].unique()

# Setup
model.eval()
all_pred_boxes = []
all_gt_boxes = []

# Collect predictions and GT boxes from all images
for filename in image_files:
    img_path = os.path.join(valid_images_dir, filename)
    image = Image.open(img_path).convert("RGB")
    img_tensor = ToTensor()(image).unsqueeze(0).to(device)

    with torch.no_grad():
        prediction = model(img_tensor)[0]

    # Filter predictions with confidence > 0.5
    pred_boxes = prediction['boxes'][prediction['scores'] > 0.5].cpu()
    all_pred_boxes.extend(pred_boxes)

    # Ground truth boxes
    gt = annotation_df[annotation_df["filename"] == filename]
    gt_boxes = torch.tensor(gt[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float)
    all_gt_boxes.extend(gt_boxes)

# Convert to tensors
all_pred_boxes = torch.stack(all_pred_boxes) if all_pred_boxes else torch.empty((0, 4))
all_gt_boxes = torch.stack(all_gt_boxes) if all_gt_boxes else torch.empty((0, 4))

# Evaluation function
def evaluate_all(pred_boxes, gt_boxes, iou_threshold=0.5):
    if len(pred_boxes) == 0 and len(gt_boxes) == 0:
        return 1, 1, 1, 1
    if len(pred_boxes) == 0 or len(gt_boxes) == 0:
        return 0, 0, 0, 0

    ious = box_iou(pred_boxes, gt_boxes)
    matched_gt = set()
    tp = 0

    for i in range(len(pred_boxes)):
        max_iou = 0
        max_j = -1
        for j in range(len(gt_boxes)):
            if j in matched_gt:
                continue
            if ious[i, j] > max_iou:
                max_iou = ious[i, j]
                max_j = j
        if max_iou >= iou_threshold:
            tp += 1
            matched_gt.add(max_j)

    fp = len(pred_boxes) - tp
    fn = len(gt_boxes) - tp

    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)
    mean_iou = ious.mean().item() if ious.numel() > 0 else 0
    return precision, recall, f1, mean_iou

# Compute overall metrics
precision, recall, f1, mean_iou = evaluate_all(all_pred_boxes, all_gt_boxes)

# Show results
print(f"📈 Overall Validation Metrics:")
print(f"   Precision: {precision:.4f}")
print(f"   Recall:    {recall:.4f}")
print(f"   F1 Score:  {f1:.4f}")
print(f"   Mean IoU:  {mean_iou:.4f}")
